# 🎅🤶 Advent of Code 2021 🌟☃️

https://adventofcode.com/

# Day 1

In [44]:
import pandas as pd
import os

def load_data():
    return pd.read_csv(os.path.join(".", "data", "1.csv"), header=None).squeeze()

### Part 1

In [44]:
def get_n_times_increased():
    return (load_data().diff().dropna() > 0.).aggregate(sum)

In [45]:
get_n_times_increased()

1527

### Part 2

In [54]:
def get_n_times_sum_increased():
    return (load_data().rolling(3).sum().diff() > 0.).aggregate(sum)

In [55]:
get_n_times_sum_increased()

1575

# Day 2

In [125]:
import pandas as pd
import os

def load_data():
    df = pd.read_csv(os.path.join(".", "data", "2.csv"), header=None)
    df.columns = ["raw"]
    df = df["raw"].str.split(expand = True)
    df.columns = ["direction", "magnitude"]
    df["magnitude"] = df["magnitude"].astype(int)
    return df

### Part 1

In [125]:
def get_product_of_horizontal_and_vertical_location():
    df = load_data()
    agg_df = df.groupby("direction").aggregate({"magnitude": sum})
    return (agg_df.loc["down", "magnitude"] - agg_df.loc["up", "magnitude"])*agg_df.loc["forward", "magnitude"]

In [126]:
get_product_of_horizontal_and_vertical_location()

1762050

### Part 2

In [123]:
def get_product_of_horizontal_and_vertical_location():
    df = load_data()
    for direction in df["direction"].unique():
        df[direction] = (df["direction"] == direction)*df["magnitude"] # change in each direction at each time step
    df["aim"] = df["down"].cumsum() - df["up"].cumsum()
    df["horizontal"] = df["forward"].cumsum()
    df["depth"] = (df["aim"]*df["forward"]).cumsum()
    return (df["depth"]*df["horizontal"]).values[-1]

In [124]:
get_product_of_horizontal_and_vertical_location()

1855892637

# Day 3

### Part 1

In [198]:
import os

def load_data():
    with open(os.path.join(".", "data", "3.csv")) as file:
        lines = file.readlines()
    df = pd.DataFrame([list(l) for l in lines]).drop(12, axis=1) # drop newline column
    return df
    
def get_power_consumption():
    df = load_data()
    gamma_rate = "".join(df.mode(axis=0).transpose().squeeze().values[:-1])
    epsilon_rate = "".join([{"0":"1", "1": "0"}[c] for c in gamma_rate])
    return int(gamma_rate, base=2)*int(epsilon_rate, base=2)

In [199]:
get_power_consumption()

325902

### Part 2

In [221]:
def get_life_support_rating():
    def _filter_by_bit_criterion(numbers_df, criterion, bit_i=0,):
        if criterion == "O2":
            numbers_df = numbers_df.loc[numbers_df[bit_i] == max(numbers_df[bit_i].mode()), :]
        elif criterion == "CO2":
            numbers_df = numbers_df.loc[numbers_df[bit_i] != max(numbers_df[bit_i].mode()), :]
        else:
            raise ValueError()
        
        if numbers_df.shape[0] == 1:
            return int("".join(numbers_df.iloc[0, :].transpose().squeeze().values), base=2)
        
        else:
            return _filter_by_bit_criterion(numbers_df, criterion, bit_i + 1)

    co2_scrubber_rating = _filter_by_bit_criterion(load_data(), criterion = "CO2")
    oxygen_generator_rating = _filter_by_bit_criterion(load_data(), criterion = "O2")
    return co2_scrubber_rating*oxygen_generator_rating

In [222]:
get_life_support_rating()

482500

# Day 4

In [ ]:
# Want to identify winning board, and the score of that board.
# The winning board is the board which first marks a row or column from the list of numbers.
# The score of the winning board is the sum of all of its unmarked numbers, multiplied by the
# last number that was read out. 

# Initialise a set of boards.
# Receive the first number.
# Update the board base on the number, check if any of the boards have won.
# If none of the boards have won, read the second number and repeat 3.
# If a board has won, compute the winning score and identify the winning board.

In [103]:
import numpy as np
import os
import re

class BingoBoard():
    
    def __init__(self, raw_board: list):
        # raw_board is a length-n list of str,
        # where each str contains n space-separated integers.
        self.values = np.array([line.split() for line in raw_board], dtype = int)
        self.marks = np.zeros(self.values.shape, dtype = bool)
        self.n = self.values.shape[0]
        self.in_play = True
    
    def update(self, number: int):
        if self.in_play:
            self.marks[(self.values == number)] = True
        
    def check_for_win(self, number: int):
        if self.in_play and (np.any(self.marks.sum(axis = 0) == self.n) or
                             np.any((self.marks.sum(axis = 1) == self.n))):
            self.in_play = False
            return np.sum(self.values[~self.marks])*number
        else:
            return False
        
def read_bingo_game(filepath: str):
    with open(filepath) as file:
        lines = file.readlines()
    boards = []
    current_board = []
    for j, line in enumerate(lines):
        if j == 0: # read list of called-out numbers
            called_values = np.array(line.strip().split(","), dtype = int)
        
        if j > 1: # read boards
            if (re.search("[0-9]+", line) is not None):
                current_board.append(line.strip())
            else:
                boards.append(BingoBoard(current_board))
                current_board = []
            
    return called_values, boards

def evaluate_bingo_game(called_values: list, boards: list):
    # called values is a list of integers.
    # boards is a list of BingoBoards.
    outcomes = [] # list of win indicators/board final scores
    # round of play increases going down the rows,
    # columns correspond to boards.
    for called_value in called_values:
        for board in boards:
            board.update(called_value)
        outcomes.append([board.check_for_win(called_value) for board in boards])
    return np.array(outcomes)

### Part 1

In [109]:
# First win
def get_first_win():
    called_values, boards = read_bingo_game(os.path.join(".", "data", "4.csv"))
    outcomes = evaluate_bingo_game(called_values, boards)
    n_rounds = outcomes.shape[0]
    for j in range(n_rounds):
        max_outcome = max(outcomes[j, :])
        if max_outcome > 0:
            return max_outcome
    

In [110]:
get_first_win()

49686

### Part 2

In [113]:
# Last win
def get_last_win():
    called_values, boards = read_bingo_game(os.path.join(".", "data", "4.csv"))
    outcomes = evaluate_bingo_game(called_values, boards)
    n_rounds = outcomes.shape[0]
    for j in range(outcomes.shape[0]):
        max_outcome = max(outcomes[n_rounds - 1 - j, :])
        if max_outcome > 0:
            return max_outcome

In [114]:
get_last_win()

26878

# Day 5

### Part 1

In [179]:
import pandas as pd
import numpy as np
import os

In [280]:
EPS = 1e-10

def transform_vent_coords_to_points(vent_coords: str, ignore_diagonals = True):
    origin, terminus = [
        np.array(coords.split(","), dtype = int)
            for coords in re.findall(r"[0-9,]+", vent_coords)
    ]
    magnitude = np.sqrt(np.sum((terminus - origin)**2.))
    direction = np.expand_dims((terminus - origin)/magnitude, axis = -1)
    if np.all(np.abs(direction) > 0.7): # diagonal line
        if ignore_diagonals:
            return None
        points = np.expand_dims(origin, axis = -1) + direction*np.arange(0., magnitude + EPS, np.sqrt(2))
    else: # horizontal or vertical line
        points = np.expand_dims(origin, axis = -1) + direction*np.arange(0., magnitude + EPS, 1.)
    return np.rint(points).transpose()

In [281]:
def analyse_vent_coords_collection(ignore_diagonals = True):
    with open(os.path.join(".", "data", "5.csv")) as file:
        lines = file.readlines()
    
    vent_points = [transform_vent_coords_to_points(vc, ignore_diagonals) for vc in lines]
    vent_points = np.concatenate([vp for vp in vent_points if vp is not None])
    
    # Concatenate all vent points
    return pd.Series([tuple(vent_point) for vent_point in vent_points]).value_counts()

In [282]:
np.sum(analyse_vent_coords_collection(ignore_diagonals = True) >= 2)

5169

In [283]:
np.sum(analyse_vent_coords_collection(ignore_diagonals = False) >= 2)

22083

# Day 6

In [284]:
# Each lanternfish creates a new lanternfish once every 7 days.
# After being born, a lanternfish has a timer value of 8.
# After giving birth, a lanternfish has a timer value of 6.
# The lanternfish gives birth in the interval between timer values of 0 and 6.

# How many lanternfish would there be after 80 days?

### Part 1

In [300]:
import numpy as np
import pandas as pd
import os

In [301]:
def load_lanternfish_data():
    with open(os.path.join(".", "data", "6.csv")) as file:
        data = np.array(file.readlines()[0].strip().split(","), dtype = int)
    return data

def increment_lanternfish_data(data, n_days = 1):
    # data is a 1d np.array of integers.
    giving_birth = (data == 0)
    data[giving_birth] = 6
    data[~giving_birth] = data[~giving_birth] - 1
    data = np.concatenate([data, np.full(shape = giving_birth.sum(), fill_value = 8)])
    if (n_days - 1) == 0:
        return data
    else:
        return increment_lanternfish_data(data, n_days - 1)

In [302]:
len(increment_lanternfish_data(load_lanternfish_data(), n_days = 80))

362639

### Part 2

In [ ]:
# This part of the question asks us to compute the number of lanternfish
# after 256 days. Doing this using the existing code would require too much memory.
# Instead, we can keep track of the count of the number of lanternfish with k
# days until giving birth. This will mean that the memory footprint of the lanternfish
# tracker does not increase in proportion to the number of days.

In [315]:
def load_lanternfish_data_v2():
    with open(os.path.join(".", "data", "6.csv")) as file:
        data = np.array(file.readlines()[0].strip().split(","), dtype = int)
    data = pd.Series(data).value_counts()
    for j in set(range(0, 9)) - set(data.index.values):
        data[j] = 0
    return data
    
def increment_lanternfish_data_v2(data, n_days = 1):
    n_gave_birth = data[0]
    for j in range(0, 8):
        data[j] = data[j + 1]
    data[8] = n_gave_birth
    data[6] += n_gave_birth
    if (n_days - 1) == 0:
        return data.sum()
    else:
        return increment_lanternfish_data_v2(data, n_days - 1)

In [316]:
increment_lanternfish_data_v2(load_lanternfish_data_v2(), n_days = 80)

362639

In [317]:
increment_lanternfish_data_v2(load_lanternfish_data_v2(), n_days = 256)

1639854996917

# Day 7

### Part 1

In [319]:
# Given x1, x2, ..., xn,
# minimize sum_i(|xi - a|)
# w.r.t. a.
# The solution to this problem is the median.

In [329]:
import numpy as np

In [326]:
def load_crab_data():
    with open(os.path.join(".", "data", "7.csv")) as file:
        data = np.array(file.readlines()[0].split(","), dtype=float)
    return data

In [328]:
np.sum(np.abs(load_crab_data() - np.median(load_crab_data())))

340056.0

### Part 2

In [330]:
from scipy.optimize import minimize_scalar

In [336]:
# Given x1, x2, ..., xn
# minimize sum_i(0.5*|xi - a|(|xi - a| + 1))
# w.r.t. a

def cost_function(a: float, data: np.array):
    return np.sum(0.5*np.abs(data - a)*(np.abs(data - a) + 1))

def optimize_crab_location():
    data = load_crab_data()
    a_star = 0.
    min_cost = np.inf
    for a_dash in np.arange(np.min(data), np.max(data) + 1.):
        cost_dash = cost_function(a_dash, data)
        if  cost_dash < min_cost:
            min_cost = cost_dash
            a_star = a_dash
    return a_star, min_cost

In [337]:
optimize_crab_location()

(460.0, 96592275.0)

# Day 8

### Part 1

In [345]:
# Each display has segments a-g and display a number 0-9.
# You make a note of all ten unique signal patterns you see,
# and then write down a single four digit output value.
# Use the signal patterns to figure out which pattern
# corresponds to which digit, then decode the four-digit output
# value.
# Initially focus on easy digits - 1, 4, 7, 8

def load_segments_data():
    with open(os.path.join(".", "data", "8.csv")) as file:
        lines = file.readlines()
    signal_patterns, output_values = list(zip(*[line.split("|") for line in lines]))
    signal_patterns = [pattern.strip().split() for pattern in signal_patterns]
    output_values = [output_value.strip().split() for output_value in output_values]
    return signal_patterns, output_values

In [349]:
# In the output values, how many times do digits 1, 4, 7, or 8 appear?

def count_1_4_7_8_in_output_values():
    _, output_values = load_segments_data()
    return sum([count_1_4_7_8_in_output_value(output_value) for output_value in output_values])
        
def count_1_4_7_8_in_output_value(output_value: list):
    return sum([len(entry) in [2, 4, 3, 7] for entry in output_value])

In [350]:
count_1_4_7_8_in_output_values()

539

### Part 2

In [427]:
# Brute force solution - try each possible mapping.
from itertools import permutations

digit_to_segments = {
    0: ["A", "B", "C", "E", "F", "G"],
    1: ["C", "F"],
    2: ["A", "C", "D", "E", "G"],
    3: ["A", "C", "D", "F", "G"],
    4: ["B", "C", "D", "F"],
    5: ["A", "B", "D", "F", "G"],
    6: ["A", "B", "D", "E", "F", "G"],
    7: ["A", "C", "F"],
    8: ["A", "B", "C", "D", "E", "F", "G"],
    9: ["A", "B", "C", "D", "F", "G"]
}
segments_to_digit = {
    "".join(v): str(k) for k, v in digit_to_segments.items()
}

def decode_wire_to_segment_mapping(signal_pattern: list):
    # There is a smarter way to do this but this way is easier to implement.
    wires = [wire for wire in "abcdefg"]
    valid_segment_illuminations = [
        illuminated_segments for illuminated_segments in digit_to_segments.values()
    ]
    # Search over all possible wire-to-segment mappings.
    for segment_order in permutations([segment for segment in "ABCDEFG"]):
        M = {wire: segment for wire, segment in zip(wires, segment_order)}
        valid_mapping = True
        
        for digit_str in signal_pattern:
            illuminated_segments = sorted([M[wire] for wire in digit_str])
            
            # If mapping generates nonsensical illumination, move on
            # to next candidate mapping M.
            if illuminated_segments not in valid_segment_illuminations:
                valid_mapping = False
                break
        
        if valid_mapping:
            return M
            # If not, move on to next digit str
            
def decode_output_value_from_signal_pattern(signal_pattern: list, output_value: list):
    wire_to_segment = decode_wire_to_segment_mapping(signal_pattern)
    
    def _map_output_value_to_digits(output_value: list, wire_to_segment: dict):
        digits = []
        for digit_str in output_value:
            segments = "".join(sorted([wire_to_segment[wire] for wire in digit_str]))
            digits.append(segments_to_digit[segments])
        
        return int("".join(digits))
    
    return _map_output_value_to_digits(output_value, wire_to_segment)

In [428]:
signal_patterns, output_values = load_segments_data()

sum([decode_output_value_from_signal_pattern(signal_pattern, output_value)
        for signal_pattern, output_value in zip(signal_patterns, output_values)])

1084606

# Day 9

### Part 1

In [6]:
# 2199943210
# 3987894921
# 9856789892
# 8767896789
# 9899965678

# Numbers are heights.
# Find all the low points,
# where a low point is a point with a value smaller
# then its (up to) four adjacent points.
# The risk level of a low point is 1 + its height.

import numpy as np
import os

def load_height_data():
    with open(os.path.join(".", "data", "9.csv")) as file:
        lines = file.readlines()
    return np.array([[int(h) for h in line.strip()] for line in lines])

def identify_low_points(height_data: np.array):
    # Could vectorize this.
    low_points_grid = np.full(height_data.shape, fill_value = False)
    height_data = np.pad(height_data, pad_width = 1, constant_values = 10) 
    for row_i in range(low_points_grid.shape[0]):
        for col_i in range(low_points_grid.shape[1]):
            low_points_grid[row_i, col_i] = np.all([
                (height_data[row_i + 1, col_i + 1] - \
                    height_data[row_i + 1 + off_r, col_i + 1 + off_c]) < 0
                for off_r, off_c in [(-1, 0), (1, 0), (0, 1), (0, -1)]
            ])
    
    return low_points_grid

def compute_risk_of_height_data(height_data: np.array):
    low_points = identify_low_points(height_data)
    return np.sum(height_data[low_points] + 1)

In [7]:
test_data = np.array([[int(h) for h in line] for line in """2199943210
3987894921
9856789892
8767896789
9899965678""".split("\n")])

In [8]:
compute_risk_of_height_data(test_data)

15

In [9]:
compute_risk_of_height_data(load_height_data())

480

### Part 2

In [31]:
from itertools import product
import numpy as np

In [82]:
def get_product_of_three_largest_basin_sizes(height_data: np.array):
    # An edge (a, b) exists if a and b are adjacent and neither of them are equal to 9.
    # Iterate over nodes, testing edges to below and to the right.
    edges = []
    for row_i in range(height_data.shape[0] - 1):
        for col_i in range(height_data.shape[1] - 1):
            for off_r, off_c in [(0, 1), (1, 0)]:
                if height_data[row_i, col_i] != 9 and height_data[row_i + off_r, col_i + off_c] != 9:
                    edges.append([(row_i, col_i), (row_i + off_r, col_i + off_c)])
                    
    # With this list of edges, create sets of nodes in each
    # basin. A basin is a set of nodes.
    # Iterate over edges. Take edge (a, b).
    # Let Sa be the basin containing a, or {a} if this basin does not exist.
    # Let Sb be the basin containing b, or {b} if this basin does not exist.
    # Take the union of Sa and Sb.
    basins = [{node} for node in list(set(node for edge in edges for node in edge))]
    for node_a, node_b in edges:
        # Get basin indices
        Sa_i = np.argwhere([node_a in basin for basin in basins]).squeeze()
        Sb_i = np.argwhere([node_b in basin for basin in basins]).squeeze()
        
        # Connect basins by merging them
        basins[Sa_i] = basins[Sa_i].union(basins[Sb_i])
        if Sa_i != Sb_i:
            basins.pop(Sb_i)
    
    basin_sizes = sorted([len(basin) for basin in basins])
    
    return np.prod(basin_sizes[-3:])

In [83]:
get_product_of_three_largest_basin_sizes(load_height_data())

1045660

# Day 10

In [ ]:
# Corrupted line = Line that has an equal number of opening and closing braces,
# but these braces do not match.
# Incomplete line = Line that does not have an equal number of opening and closing braces.

# Find the first illegal character in each corrupted line of the navigation subsystem. What is the total syntax error score for those errors?

In [112]:
import numpy as np
import os

In [113]:
def load_syntax_error_data():
    with open(os.path.join(".", "data", "10.csv")) as file:
        lines = [line.strip() for line in file.readlines()]
    return lines

### Part 1

In [114]:
# Iterate over characters of line.
# If opening bracket, push to end of list of open brackets.
# If closing bracket, check it matches the final entry in the list of open brackets.
#    If it does not, raise a syntax error.
#    If it does, pop the final entry in the list of open brackets.

def find_first_syntax_error(line: str):
    # Returns None if no syntax errors raised,
    # illegal character otherwise.
    close_to_open = {
        "]": "[",
        ")": "(",
        "}": "{",
        ">": "<"
    }
    open_brackets = ""
    for c in line:
        if c in "([{<":
            open_brackets += c
        elif c in ")]}>":
            if close_to_open[c] == open_brackets[-1]: # Correctly closes a chunk.
                open_brackets = open_brackets[:-1]
            else: # Does not correctly close a chunk.
                return c
            
def compute_total_syntax_error_score():
    char_to_points = {
        ")": 3,
        "]": 57,
        "}": 1197,
        ">": 25137
    }
    illegal_chars = [find_first_syntax_error(line) for line in load_syntax_error_data()]
    return sum([char_to_points[c] for c in illegal_chars if c is not None])
    

In [115]:
test_line = "{([(<{}[<>[]}>{[]{[(<()>"
check_syntax(test_line)

'}'

In [116]:
compute_total_syntax_error_score()

319329

### Part 2

In [117]:
def complete_line(line: str):
    close_to_open = {
        "]": "[",
        ")": "(",
        "}": "{",
        ">": "<"
    }
    open_to_close = {v: k for k, v in close_to_open.items()}
    open_brackets = []
    for c in line:
        if c in "([{<":
            open_brackets += c
        elif c in ")]}>":
            if close_to_open[c] == open_brackets[-1]: # Correctly closes a chunk.
                open_brackets = open_brackets[:-1]
    
    # Reverse remaining open brackets, map to closing brackets
    return "".join([open_to_close[c] for c in open_brackets[::-1]])

def score_autocompletion(line: str):
    # line is a sequence of closing brackets
    # Start with a total score of 0.
    # Then, for each character, multiply the total score by 5 and then increase the total
    # score by the point value given for the character in the following table:
    char_to_points = {
        ")": 1,
        "]": 2,
        "}": 3,
        ">": 4
    }
    total_score = 0
    for c in line:
        total_score = total_score*5 + char_to_points[c]
    return total_score


In [118]:
test_line = "[({(<(())[]>[[{[]{<()<>>" # - Complete by adding }}]])})].
complete_line(test_line)

'}}]])})]'

In [119]:
score_autocompletion("])}>") # 294

294

In [124]:
# Discard corrupted lines
def compute_middle_autocomplete_score():
    incomplete_lines = [line for line in load_syntax_error_data() if find_first_syntax_error(line) is None]
    autocompletions = [complete_line(line) for line in incomplete_lines]
    return np.median([score_autocompletion(ac) for ac in autocompletions])

In [123]:
compute_middle_autocomplete_score()

3515583998.0

# Day 11

### Part 1

In [ ]:
# First, the energy level of each octopus increases by 1
# Then, any octopus with an energy level greater than 9 flashes.
# This increases the energy level of all adjacent octopuses by 1, including
# octopuses that are diagonally adjacent. If this causes an octopus to have
# an energy level greater than 9, it also flashes. This process continues as 
# long as new octopuses keep having their energy level increased beyond 9.
# (An octopus can only flash at most once per step.)
# Finally, any octopus that flashed during this step has its energy level set
# to 0, as it used all of its energy to flash.


In [196]:
import os
import numpy as np

def load_octopus_data(test = False):
    if test:
        lines = """5483143223
                    2745854711
                    5264556173
                    6141336146
                    6357385478
                    4167524645
                    2176841721
                    6882881134
                    4846848554
                    5283751526""".split("\n")
    else:
        with open(os.path.join(".", "data", "11.csv")) as file:
            lines = file.readlines()
    return np.array([[int(c) for c in line.strip()] for line in lines])

def iterate_octopus_data(energy_levels: np.array, n_iter = 1):
    energy_levels = energy_levels + 1
    
    def _propagate_flash(energy_levels: np.array):        
        if np.sum(energy_levels > 9) == 0:
            return energy_levels
        else:
            flashing_octopi = (energy_levels > 9)
            energy_levels[flashing_octopi] = 0
            flashing_octopi_padded = np.pad(flashing_octopi, pad_width = 1, constant_values = False)
            for row_i, col_i in np.argwhere(energy_levels > 0): 
                energy_levels[row_i, col_i] += np.sum(flashing_octopi_padded[row_i:(row_i + 3), col_i:(col_i + 3)])
            
            return _propagate_flash(energy_levels)
    
    energy_levels = _propagate_flash(energy_levels)
    if n_iter == 1:
        return energy_levels
    else:
        return iterate_octopus_data(energy_levels, n_iter = n_iter - 1)

In [184]:
def compute_total_flashes(n_iter = 100, test = False):
    total_flashes = 0
    energy_levels = load_octopus_data(test)
    for j in range(n_iter):
        energy_levels = iterate_octopus_data(energy_levels)
        total_flashes += np.sum(energy_levels == 0)
    
    return total_flashes

In [185]:
compute_total_flashes(test = True)

1656

In [186]:
compute_total_flashes(test = False)

1642

### Part 2

In [192]:
def compute_first_step_of_simultaneous_flash(test = False):
    energy_levels = load_octopus_data(test)
    n_octopi = energy_levels.shape[0]*energy_levels.shape[1]
    step = 0
    while True:
        if np.sum(energy_levels == 0) == n_octopi:
            return step
        else:
            energy_levels = iterate_octopus_data(energy_levels)
            step += 1

In [194]:
compute_first_step_of_simultaneous_flash(test = True)

195

In [195]:
compute_first_step_of_simultaneous_flash(test = False)

320

# Day 12

### Part 1

In [204]:
my_list = [1, 2, 3]
my_list.remove(2)
my_list

[1, 3]

In [213]:
import os

def load_paths_data(test = False):
    if test:
        lines = """dc-end
                    HN-start
                    start-kj
                    dc-start
                    dc-HN
                    LN-dc
                    HN-end
                    kj-sa
                    kj-HN
                    kj-dc""".split("\n")
    else:
        with open(os.path.join(".", "data", "12.csv")) as file:
            lines = file.readlines()
    edges = [line.strip().split("-") for line in lines]
    nodes = list(set(node for edge in edges for node in edge))
    
    def _get_connected_nodes(src_node, edges):
        connected_nodes = list(set(node for edge in edges for node in edge if src_node in edge))
        connected_nodes.remove(src_node)
        return connected_nodes
    
    return {node: _get_connected_nodes(node, edges) for node in nodes}

In [242]:
# Build a tree with a root at "start" and leaves at "end".

# Say we have a start node.
# Generate each possible path from that start node.
# For each generated path that does not terminate in "end",
# generate each possible path to another node.
# Terminate when no paths do not terminate in "end".


def extend_paths(connected_nodes: dict, paths=[["start"]]):
    updated_paths = []
    for path in paths:
        if path[-1] != "end": # Path needs extending
            lowercase_nodes_of_path = [n for n in path if n.islower()]
            has_visited_lowercase_node_twice = np.any(np.unique(lowercase_nodes_of_path, return_counts = True)[1] == 2)
            # has_visited_lowercase_node_twice = True # for part 1 soln
            for node in connected_nodes[path[-1]]:
                if node == "start":
                    pass
                elif has_visited_lowercase_node_twice:
                    if (node not in lowercase_nodes_of_path):
                        updated_paths.append(path + [node])
                        # NB path is dropped if no movement is possible
                else:
                    updated_paths.append(path + [node])
                
        else: # Path is already complete
            updated_paths.append(path)
    
    return updated_paths

def find_paths(connected_nodes: dict):
    paths = [["start"]]
    extended_paths = extend_paths(connected_nodes, paths)
    while extended_paths != paths:
        paths = extended_paths
        extended_paths = extend_paths(connected_nodes, paths)
    return paths

In [243]:
extend_paths(load_paths_data(test = True))

[['start', 'dc'], ['start', 'HN'], ['start', 'kj']]

In [244]:
len(find_paths(load_paths_data(test = True)))

103

In [245]:
len(find_paths(load_paths_data(test = False)))

150004

# Day 13

In [613]:
import os
import numpy as np
import re

def load_folds_data(test = False):
    if not test:
        with open(os.path.join(".", "data", "13.csv")) as file:
            lines = file.readlines()
    else:
        lines = """
            6,10
            0,14
            9,10
            0,3
            10,4
            4,11
            6,0
            6,12
            4,1
            0,13
            10,12
            3,4
            3,0
            8,4
            1,10
            2,14
            8,10
            9,0

            fold along y=7
            fold along x=5
        """.split("\n")
    
    lines = [line for line in lines if len(line.strip()) > 0] # Remove empty lines
    mark_coords = np.array([line.strip().split(",") for line in lines if "fold" not in line]).astype(int)
    fold_instructions = [re.search("([xy])=([0-9]+)", line.strip()).group(1, 2) for line in lines if "," not in line]
    fold_instructions = [[axis, int(location)] for axis, location in fold_instructions]
    return mark_coords, fold_instructions

In [652]:

def fold(mark_coords: np.array, fold_instruction):
    fold_axis, fold_location = fold_instruction
    j = {"x": 0, "y": 1}[fold_axis]
    width = max(mark_coords[:, j])
    beyond_fold = (mark_coords[:, j] > fold_location)
    before_fold = (mark_coords[:, j] < fold_location)
    if fold_location >= width/2:
        offset = width - 2*(width - fold_location)
        mark_coords[beyond_fold, j] = offset + (width - mark_coords[beyond_fold, j])
    if fold_location < width/2:
        mark_coords[beyond_fold, j] = (width - mark_coords[beyond_fold, j])
        mark_coords[before_fold, j] = (width - fold_location*2) + mark_coords[before_fold, j]
    return np.unique(mark_coords, axis = 0)

def display_marks(mark_coords: np.array):
    marks = np.full(np.max(mark_coords, axis=0)[::-1] + 1, fill_value = ".")
    for coord in mark_coords:
        marks[coord[1], coord[0]] = "#"
    return "\n".join(["".join(row) for row in marks])

In [653]:
mark_coords, fold_instructions = load_folds_data(test = True)

In [656]:
print(display_marks(fold(fold(mark_coords, fold_instructions[0]), fold_instructions[1])))

#####
#...#
#...#
#...#
#####


In [657]:
def compute_n_marks_visible_after_first_fold():
    mark_coords, fold_instructions = load_folds_data()
    mark_coords = fold(mark_coords, fold_instructions[0])
    return mark_coords.shape[0]

In [658]:
compute_n_marks_visible_after_first_fold()

837

### Part 2

In [659]:
def execute_all_fold_instructions():
    mark_coords, fold_instructions = load_folds_data()
    for fold_instruction in fold_instructions:
        mark_coords = fold(mark_coords, fold_instruction)
    return mark_coords

In [660]:
print(display_marks(execute_all_fold_instructions()))

####.###..####..##..#..#..##..#..#.#..#
#....#..#....#.#..#.#.#..#..#.#..#.#..#
###..#..#...#..#....##...#....####.#..#
#....###...#...#.##.#.#..#....#..#.#..#
#....#....#....#..#.#.#..#..#.#..#.#..#
####.#....####..###.#..#..##..#..#..##.


In [ ]:
# EPZGKCHU

# Day 14

In [677]:
import os
import re

def load_polymer_data(test = False):
    if test:
        lines = """NNCB
                CH -> B
                HH -> N
                CB -> H
                NH -> C
                HB -> C
                HC -> B
                HN -> C
                NN -> C
                BH -> H
                NC -> B
                NB -> B
                BN -> B
                BB -> N
                BC -> B
                CC -> N
                CN -> C
                """.split("\n")
    else:
        with open(os.path.join(".", "data", "14.csv")) as file:
            lines = file.readlines()
    
    return lines[0].strip(), [re.match("([A-Z]{2}) -> ([A-Z]{1})", line.strip()).group(1, 2)
                              for line in lines[1:] if len(line.strip()) > 0]
    

### Part 1

In [705]:
def apply_instructions(polymer: str, instructions: list, n_steps = 1):
    # Extract pairs, inserted_chars
    pairs, insertions = list(zip(*instructions))
    
    # Iterate over characters of polymer
    c0 = polymer[0]
    new_polymer = polymer[0]
    for j in range(1, len(polymer)):
        c1 = polymer[j]
        try: # Insert value if c0 + c1 is in pairs
            new_polymer = new_polymer + insertions[pairs.index(c0 + c1)] + c1
        except ValueError: # If c0 + c1 not in pairs
            new_polymer = new_polymer + c1
        c0 = c1
        
    if n_steps == 1:
        return new_polymer
    else:
        return apply_instructions(new_polymer, instructions, n_steps = n_steps - 1)

In [707]:
polymer, instructions = load_polymer_data(test = True)
apply_instructions(polymer, instructions, n_steps = 2)

'NBCCNBBBCBHCB'

In [713]:
np.unique([c for c in "aabbc"], return_counts = True)[1]

array([2, 2, 1], dtype=int64)

In [742]:
def n_most_common_char_minus_n_least_common_char(s: str):
    char_counts = np.unique([c for c in s], return_counts = True)[1]
    return max(char_counts) - min(char_counts)

### Part 2

In [743]:
def polymer_to_pair_counts(polymer: str):
    pair_counts = {}
    for j in range(1, len(polymer)):
        pair = polymer[(j-1):(j+1)]
        pair_counts[pair] = pair_counts.get(pair, 0) + 1
    return pair_counts

In [763]:
def apply_instructions_v2(pair_counts: dict, instructions: list, n_steps = 1):
    # Iterate over pairs in pair_counts
    # If pair in insertions, update pair_counts
    pairs, insertions = list(zip(*instructions))
    for pair, n in list(pair_counts.items()):
        try: # In instructions
            inserted_c = insertions[pairs.index(pair)]
            pair_counts[pair[0] + inserted_c] = pair_counts.get(pair[0] + inserted_c, 0) + n
            pair_counts[inserted_c + pair[1]] = pair_counts.get(inserted_c + pair[1], 0) + n
            pair_counts[pair] = pair_counts[pair] - n
        except ValueError:
            pass
    
    if n_steps == 1:
        return pair_counts
    else:
        return apply_instructions_v2(pair_counts, instructions, n_steps = n_steps - 1)

In [764]:
def pair_counts_to_char_counts(pair_counts: dict, first_char, last_char):
    char_counts = {}
    for pair, count in pair_counts.items():
        for c in pair:
            char_counts[c] = char_counts.get(c, 0) + count
            
    char_counts[first_char] += 1
    char_counts[last_char] += 1
    char_counts = {k: v//2 for k, v in char_counts.items()}
    return char_counts

In [765]:
def n_most_common_char_minus_n_least_common_char_v2(polymer: str, instructions: list, n_steps: int):
    pair_counts = polymer_to_pair_counts(polymer)
    pair_counts = apply_instructions_v2(pair_counts, instructions, n_steps)
    char_counts = pair_counts_to_char_counts(pair_counts, polymer[0], polymer[1])
    return max(char_counts.values()) - min(char_counts.values())

In [767]:
polymer, instructions = load_polymer_data(test = False)
n_most_common_char_minus_n_least_common_char_v2(polymer, instructions, n_steps = 40)

2265039461737

# Day 15

In [998]:
def load_mollusc_data(test = False):
    if test:
        lines = """
            1163751742
            1381373672
            2136511328
            3694931569
            7463417111
            1319128137
            1359912421
            3125421639
            1293138521
            2311944581
        """.split("\n")
    else:
        with open(os.path.join(".", "data", "15.csv")) as file:
            lines = file.readlines()
    
    return np.array([[c for c in line.strip()] for line in lines if len(line.strip()) > 0]).astype(int)

In [1073]:
import numpy as np
from itertools import product

def find_shortest_path(node_costs, start_node):
    """ Implementation of a modified version of Dijkstra's algorithm for finding the shortest path
        through a weighted graph.
    """
    def _node_neighbours(node: tuple, node_costs: np.array, unvisited_nodes: set):
        return [(neighbour, node_costs[neighbour[0], neighbour[1]])
                for neighbour in [(node[0] - 1, node[1]), (node[0], node[1] - 1), (node[0], node[1] + 1), (node[0] + 1, node[1])]  
                if neighbour in unvisited_nodes]
                
        return node_neighbours
    
    def _get_current_node(tentative_distances: dict, n_rows, n_cols):
        # return min(tentative_distances, key = tentative_distances.get) # Dijkstra's algorithm
        # tentative_distances contains upper bounds on the minimum distance to each unvisited node.
        # below uses A* cost with an admissible heuristic function.
        return min(tentative_distances, key = lambda k: tentative_distances[k] + (n_rows - 1 - k[0] + n_cols - 1 - k[1]))
    
    n_rows, n_cols = node_costs.shape
    nodes = list(product(range(n_rows), range(n_cols)))
    end_node = tuple(np.max(nodes, axis = 0).astype(int))
    unvisited_nodes = set(nodes)
    #tentative_distances = {node: 9*(node[0] + node[1]) for node in nodes}
    tentative_distances = {}
    tentative_distances[start_node] = 0
    current_node = start_node
    
    while True:
        # Iterate over neighbours of current node.
        for neighbour, distance in _node_neighbours(current_node, node_costs, unvisited_nodes):
            # The most promising node is the one with smallest minimum tentative distance to the end,
            # not the smallest minimum tentative distance to the start.
            distance_from_start_to_neighbour_via_current_node = tentative_distances[current_node] + distance
            if distance_from_start_to_neighbour_via_current_node < tentative_distances.get(neighbour, np.inf):
                tentative_distances[neighbour] = distance_from_start_to_neighbour_via_current_node
        
        # Break if the end node has been visited.
        if current_node == end_node:
            break
            
        # Mark the current node as visited.
        unvisited_nodes.remove(current_node)
        del tentative_distances[current_node]
        
        # Set current node to unvisited node with smallest tentative distance.
        current_node = _get_current_node(tentative_distances, n_rows, n_cols)
    
    return tentative_distances[end_node]

In [1074]:
find_shortest_path(load_mollusc_data(True), (0, 0))

40

In [1075]:
find_shortest_path(load_mollusc_data(False), (0, 0))

562

### Part 2

In [1078]:
def expand_node_costs(node_costs: np.array):
    nrow, ncol = node_costs.shape
    expanded_node_costs = np.zeros([nrow*5, ncol*5])
    for row_i in range(5):
        for col_i in range(5):
            tile = node_costs + row_i + col_i
            mask = tile > 9
            tile[mask] = np.mod(tile[mask], 10) + 1
            expanded_node_costs[(row_i*nrow):((row_i + 1)*nrow), (col_i*ncol):((col_i + 1)*ncol)] = tile
    return expanded_node_costs.astype(int)

In [1079]:
find_shortest_path(expand_node_costs(load_mollusc_data(True)), start_node = (0, 0))

315

In [1080]:
find_shortest_path(expand_node_costs(load_mollusc_data(False)), start_node = (0, 0))

2874

# Day 16

### Part 1

In [1211]:
import os

def load_bits_data():
    with open(os.path.join(".", "data", "16.csv")) as file:
        lines = file.readlines()
    data_hex = [line.strip() for line in lines if len(line) > 0][0]
    
    def _hex_to_bin(c: str):
        return str(bin(int(c, 16)))[2:].zfill(4)

    data_bin = "".join([_hex_to_bin(c) for c in data_hex])
    return data_bin

In [1212]:
def get_packet_version(data_bin: str):
    return int(data_bin[0:3], 2), data_bin[3:]

def get_packet_type_id(data_bin: str):
    return get_packet_version(data_bin)

def consume_binary_number_chunk(data_bin: str):
    leading_bit = data_bin[0]
    number_chunk = data_bin[1:5]
    return leading_bit, number_chunk, data_bin[5:]

def consume_literal(data_bin: str):
    parsed_binary_number = ""
    leading_bit = ""
    while leading_bit != "0":
        leading_bit, number_chunk, data_bin = consume_binary_number_chunk(data_bin)
        parsed_binary_number += number_chunk
    
    return int(parsed_binary_number, 2), data_bin.rstrip("0")

In [1213]:
def parse_packet(data_bin: str):
    packet = {}
    packet["version"], data_bin = get_packet_version(data_bin)
    packet["type_id"], data_bin = get_packet_type_id(data_bin)
    if packet["type_id"] == 4: # Literal packet
        packet["value"], data_bin = consume_literal(data_bin)
    else: # Operator packet - contains other packets
        packet_length_type_id = data_bin[0]
        data_bin = data_bin[1:]
        if packet_length_type_id == "0":
            length_of_subpackets = int(data_bin[:15], 2) # in bits
            data_bin = data_bin[15:]
            init_data_bin_length = len(data_bin)
            while ((init_data_bin_length - len(data_bin)) < length_of_subpackets) and len(data_bin) > 0:
                child, data_bin = parse_packet(data_bin)
                packet["children"] = packet.get("children", []) + [child]
        else:
            number_of_subpackets = int(data_bin[:11], 2)
            data_bin = data_bin[11:]
            while (len(packet.get("children", [])) < number_of_subpackets) and len(data_bin) > 0:
                child, data_bin = parse_packet(data_bin)
                packet["children"] = packet.get("children", []) + [child]
        
    return packet, data_bin

In [1214]:
parse_packet("11101110000000001101010000001100100000100011000001100000") # expect 7, 3, (1 2, 3)

({'version': 7,
  'type_id': 3,
  'children': [{'version': 2, 'type_id': 4, 'value': 1},
   {'version': 4, 'type_id': 4, 'value': 2},
   {'version': 1, 'type_id': 4, 'value': 3}]},
 '')

In [1215]:
parse_packet("110100101111111000101000")[0]

{'version': 6, 'type_id': 4, 'value': 2021}

In [1216]:
def add_version_number_to_total(packet: dict, version_total: int):
    version_total += int(packet["version"])
    for child in packet.get("children", []):
        version_total = add_version_number_to_total(child, version_total)
    
    return version_total

In [1217]:
add_version_number_to_total(parse_packet("11101110000000001101010000001100100000100011000001100000")[0], 0) # expect 14

14

In [1218]:
add_version_number_to_total(parse_packet(load_bits_data())[0], 0)

965

### Part 2

In [1219]:
from math import prod

In [1225]:
TYPE_ID_TO_OPERATOR = {
    0: sum,
    1: prod,
    2: min,
    3: max,
    5: lambda v: int(v[0] > v[1]),
    6: lambda v: int(v[0] < v[1]),
    7: lambda v: int(v[0] == v[1])
}

def evaluate_packet_value(packet: dict):
    # Converts a packet from a parent node to a leaf node
    # by evaluating the expression it contains.
    if "children" in packet.keys():
        return TYPE_ID_TO_OPERATOR[packet["type_id"]]([evaluate_packet_value(child) for child in packet["children"]])
    else:
        return packet["value"]

In [1226]:
evaluate_packet_value(parse_packet("11101110000000001101010000001100100000100011000001100000")[0])

3